In [7]:
from PIL import Image
import numpy as np
from os import path


ModuleNotFoundError: No module named 'resampling'

In [9]:
def image_to_np_array(image_name: str) -> np.array:
    img_src = Image.open(path.join('input', image_name)).convert('RGB')
    return np.array(img_src)

In [10]:

def one_step_resampling(img: np.array, factor: float, f1, f2):
    dimensions = img.shape[0:2]
    new_dimensions = tuple(f1(dimension, factor) for dimension in
                           dimensions)
    new_shape = (*new_dimensions, img.shape[2])
    new_img = np.empty(new_shape)

    for x in range(new_dimensions[0]):
        for y in range(new_dimensions[1]):
            new_img[x, y] = img[
                min(f2(x, factor), dimensions[0] - 1),
                min(f2(y, factor), dimensions[1] - 1)
            ]
    return new_img

In [11]:

def two_step_resampling(img: np.array, numerator: int,
                        denominator: int) -> np.array:
    tmp = one_step_resampling(img, numerator, lambda a, b: a * b, lambda a, b: int(round(a / b)))
    return one_step_resampling(tmp, denominator, lambda a, b: int(round(a / b)), lambda a, b: a * b)


In [12]:
def prompt(variants: dict):
    for number, variant in enumerate(variants.keys(), 1):
        print(f'{number} - {variant}')

    input_correct = False
    user_input = 0

    while not input_correct:
        try:
            user_input = int(input('> '))
            if user_input <= 0 or user_input > len(variants):
                raise ValueError
            input_correct = True
        except ValueError:
            print("Incorrect value")

    return dict(enumerate(variants.values(), 1))[user_input]

In [13]:

def safe_number_input(number_type: type, lower_bound=None, upper_bound=None):
    input_correct = False
    user_input = 0

    while not input_correct:
        try:
            user_input = number_type(input('> '))
            if lower_bound is not None and user_input < lower_bound:
                raise ValueError
            if upper_bound is not None and user_input > upper_bound:
                raise ValueError
            input_correct = True
        except ValueError:
            print("Incorrect value")
    return user_input


In [14]:

def execute(img, f1, f2, number_type=int):
    data_type = np.uint8
    color_model = 'RGB'

    factor = safe_number_input(number_type, 0.5)
    result = Image.fromarray(one_step_resampling(
        img, factor, f1, f2).astype(data_type), color_model)

    return result

In [15]:

images = {
#    'test': 'test.png',
   'second': 'second.png'
}


operations = {
    'Интерполяция': 'int',
    'Децимация': 'dec',
    'Двухпроходная передискретизация': 'two',
    'Однопроходная передискретизация': 'one'
}

In [16]:
# choose image from dict of images
print('choose image:')
selected_image = prompt(images)
img = image_to_np_array(selected_image)

# choose operation from dict op operations
print('choose operation:')
operation = prompt(operations)

match operation:
    case 'int':
        print('Enter the whole stretching factor')
        result = execute(img, lambda a, b: a * b, lambda a, b: int(round(a / b)))

    case 'dec':
        print('Enter the whole compression ratio')
        result = execute(img, lambda a, b: int(round(a / b)), lambda a, b: a * b)

    case 'two':
        print('Enter the whole stretching factor')
        numerator = safe_number_input(int, 1)

        print('Enter the whole compression ratio')
        denominator = safe_number_input(int, 1)

        args = [numerator, denominator]
        result = Image.fromarray(
            two_step_resampling(img, *args).astype(np.uint8),
            'RGB')

    case 'one':
        print('Enter the fractional stretch/compression ratio')
        result = execute(img, lambda a, b: int(round(a * b)),
                            lambda a, b: int(round(a / b)), float)

    case _:
        exit()

print('Результат загружен: ', selected_image)
file_name = selected_image[:-4]
result.save('output/' + file_name + '_test_' + operation + '.png' )

choose image:
1 - second
Введите корректное значение
choose operation:
1 - Интерполяция
2 - Децимация
3 - Двухпроходная передискретизация
4 - Однопроходная передискретизация
Enter the whole stretching factor
Результат загружен:  second.png
